In [1]:
import pandas as pd
import numpy as np

load_file = "Resources/purchase_data.csv"

#read and store file
purchases_table = pd.read_csv(load_file)
purchases_table.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
#count number of players (len)
number_of_players = len(purchases_table["SN"].unique())

#store total number of players in a table with title 'Total Players'
number_of_players_table = pd. DataFrame({"Total Players":[number_of_players]})

#display total number of players in the table
number_of_players_table

,Total Players
0,576


In [3]:
#count number of items with (len)
number_of_items = len(purchases_table["Item Name"].unique())

#calculate avg price of items, round value with 2 after decimal and store as string with $
average_price = "$" + str(round(purchases_table["Price"].mean(), 2))

#calculate number of purchases
number_of_purchases = purchases_table["Purchase ID"].count()

#calculate total revenue
total_revenue = "$" + str(purchases_table["Price"].sum())

#create and store items in table
summary_of_purchases = pd.DataFrame({"Number of Unique Items":[number_of_items],
                                    "Average Purchase Price":[average_price],
                                    "Number of Purchases":[number_of_purchases],
                                    "Total Revenue":[total_revenue]})
summary_of_purchases

,Number of Unique Items,Average Purchase Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [12]:
#gender demographics
#players purchased items multiple times - we need to take out the duplicates
unique_players = purchases_table.drop_duplicates(subset="SN")

#group players by gender
unique_gender_groups = unique_players.groupby("Gender")

#create summary table
#count number of unique SN in each gender group while renaming the column of SN to Total Count
gender_table = unique_gender_groups.count()[["SN"]].rename(columns={"SN":"Total Count"})
#create column of Percentage of Total Players
gender_table["Percentage of Total Players"] = round(gender_table["Total Count"]/number_of_players*100,2)
#display
gender_table

,Total Count,Percentage of Total Players
Gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


In [20]:
#calculate stats by gender
#create gender groups
gender_groups = purchases_table.groupby(["Gender"])

#count purchases by gender
total_puchase_gender = gender_groups["Purchase ID"].count()

#average purchase price per gender
avg_purchase_price_gender = gender_groups["Price"].mean().map("${:.2f}".format)

#total purchase per gender
total_purchase_gender = gender_groups["Price"].sum().map("${:,.2f}".format)

#average purchase per person = sum price divide by unique gender groups
avg_purchase_per_person = (gender_groups["Price"].sum() / 
                           unique_gender_groups["SN"].count()).map("${:.2f}".format)

#create new table from previous calculations
gender_analysis_table = pd.DataFrame({"Purchase Count":total_purchase_gender, 
                                     "Average Purchase Price":avg_purchase_price_gender, 
                                     "Total Purchase Value":total_purchase_gender, 
                                     "Avg Total Purchase per Person":avg_purchase_per_person})
gender_analysis_table

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,$361.94,$3.20,$361.94,$4.47
Male,"$1,967.64",$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,$50.19,$3.35,$50.19,$4.56


In [21]:
#categorize ages
bins = [0,9,14,19,24,29,34,39,float("inf")]
bin_names = ["<10",
             "10-14",
             "15-19",
             "20-24",
             "25-29",
             "30-34",
             "35-39",
             "40+"]

#add age bins column
unique_players["Age Group"] = pd.cut(unique_players["Age"],bins,right=True,labels=bin_names)

#group by age
age_bins = unique_players.groupby(["Age Group"])

#create table
age_demo = age_bins.count()[["SN"]].rename(columns={"SN":"Total Count"})
age_demo["Percentage of Players"] = round(age_demo["Total Count"]/number_of_players*100,2)
age_demo

/var/folders/hv/b41m52jx0_jbhnj2ns175pv80000gp/T/ipykernel_78907/1571880028.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_players["Age Group"] = pd.cut(unique_players["Age"],bins,right=True,labels=bin_names)


,Total Count,Percentage of Players
Age Group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [22]:
#bin purchases_table by age
purchases_table["Age Group"] = pd.cut(purchases_table["Age"], bins, right=True, labels=bin_names)
age_bin_group = purchases_table.groupby(["Age Group"])

#purchase count per age
purchase_age = age_bin_group["Purchase ID"].count()
avg_purchase_price_age = age_bin_group["Price"].mean().map("${:.2f}".format)
total_purchase_age = age_bin_group["Price"].sum().map("${:,.2f}".format)

#total purchase value / age-grouped counts of UNIQUE players (using groupby obj from earlier)
avg_purchase_per_person_age = (age_bin_group["Price"].sum() / 
                        age_bins["SN"].count()).map("${:.2f}".format)

#create new dataframe from calculations performed on groupby objects
age_analysis_table = pd.DataFrame({"Purchase Count":purchase_age,
                                "Average Purchase Price":avg_purchase_price_age,
                                "Total Purchase Value":total_purchase_age,
                                "Avg Total Purchase per Person":avg_purchase_per_person_age})
age_analysis_table

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [23]:
#group by player names
player_group = purchases_table.groupby("SN")

#aggregations on Purchase ID and Price columns, rename columns, and sort by TPV
top_spenders_table = player_group.agg({"Purchase ID":'count',
                                  "Price":['mean','sum']})
top_spenders_table.columns = ["Purchase Count","Average Purchase Price","Total Purchase Value"]
top_spenders_table.sort_values("Total Purchase Value",inplace=True,ascending=False)

#formatting
top_spenders_table["Average Purchase Price"] = top_spenders_table["Average Purchase Price"].map("${:.2f}".format)
top_spenders_table["Total Purchase Value"] = top_spenders_table["Total Purchase Value"].map("${:.2f}".format)

#show top 5 spenders
top_spenders_table.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.41,$13.62
Iskadarya95,3,$4.37,$13.10


In [24]:
#retrieve Item ID, Item Name, Item Price, and store
items_table = purchases_table[["Item ID","Item Name","Price"]]
items_group = items_table.groupby(["Item ID","Item Name"])

#store purchase counts, sorted
init_table = items_group.count().sort_values("Price",ascending=False).rename(columns={"Price":"Purchase Count"})

#create table of unique item IDs and their price, drop duplicates while rename price column
unique_items_table = items_table[["Item ID","Price"]].drop_duplicates(subset='Item ID').rename(columns={"Price":"Item Price"})

#merge initial with unique_items_df
merged_table = pd.merge(init_table.reset_index(),unique_items_table, on="Item ID",how='left').set_index(["Item ID","Item Name"])

#TPV = purchase count * item price. 
merged_table["Total Purchase Value"] = merged_table["Purchase Count"]*merged_table["Item Price"]
new_merged = merged_table.copy()

#clean formatting
new_merged["Item Price"] = new_merged["Item Price"].map("${:.2f}".format)
new_merged["Total Purchase Value"] = new_merged["Total Purchase Value"].map("${:.2f}".format)

#show top 5
new_merged.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$63.44
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.19,$28.71
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [25]:
#most profitable items
profitable_table = merged_table.sort_values("Total Purchase Value",ascending=False)

#clean formatting
profitable_table["Item Price"] = profitable_table["Item Price"].map("${:.2f}".format)
profitable_table["Total Purchase Value"] = profitable_table["Total Purchase Value"].map("${:.2f}".format)

#show top 5
profitable_table.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$63.44
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
